In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from early_exit_resnet import *


In [11]:
head_net_part1 = HeadNetworkPart1(block=Bottleneck, in_planes=64, num_blocks=[3], num_classes=10)
head_net_part2 = HeadNetworkPart2(Bottleneck, 256, [4], num_classes=10)
head_net_part3 = HeadNetworkPart3(block=Bottleneck, in_planes=512, num_blocks=[6], num_classes=10)
tail_net = TailNetwork(block=Bottleneck, in_planes=1024, num_blocks=[3, 4, 6, 3], num_classes=10)


In [12]:
combined_state_dict = torch.load('lightning_logs/version_14/checkpoints/epoch=14-step=4695.ckpt')


In [13]:

head1_state_dict = {}
head2_state_dict = {}
head3_state_dict = {}
tail_state_dict = {}


for key, value in combined_state_dict['state_dict'].items():
    if key.startswith('head1'):
        head1_state_dict[key.removeprefix('head1.')] = value
    elif key.startswith('head2'):
        head2_state_dict[key.removeprefix('head2.')] = value
    elif key.startswith('head3'):
        head3_state_dict[key.removeprefix('head3.')] = value
    else:
        tail_state_dict[key.removeprefix('tail.')] = value


In [14]:
torch.save(head1_state_dict,"models/head1_resnet50.pth")
torch.save(head2_state_dict,"models/head2_resnet50.pth")
torch.save(head3_state_dict,"models/head3_resnet50.pth")
torch.save(tail_state_dict,"models/tail_resnet50.pth")

In [15]:
head_net_part1.load_state_dict(torch.load("models/head1_resnet50.pth"))
head_net_part2.load_state_dict(torch.load("models/head2_resnet50.pth"))
head_net_part3.load_state_dict(torch.load("models/head3_resnet50.pth"))
tail_net.load_state_dict(torch.load("models/tail_resnet50.pth"))


<All keys matched successfully>

In [16]:
def evaluate_model(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    total_samples = len(test_loader.dataset)
    correct_predictions = [0, 0, 0]  # Assuming three exits

    with torch.no_grad():  # No need to compute gradients
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            exits = model(inputs)  # Forward pass
            softmax = torch.nn.Softmax(dim=1)
            
            for i, exit in enumerate(exits):
                predictions = softmax(exit).argmax(dim=1)
                correct_predictions[i] += (predictions == labels).type(torch.float).sum().item()

    accuracies = [correct / total_samples for correct in correct_predictions]
    return accuracies